<div>
<img src="https://cme-solution-accelerators-images.s3-us-west-2.amazonaws.com/toxicity/solution-accelerator-logo.png"; width="50%">
</div>


# About This Series of Notebooks

This series of notebooks is intended to help you use AI_Query functions in Databricks and identify common sentiment and features from your customer feedback

In support of this goal, we will:

- Load customer feedback data from Amazon
- Use out of the box [AI functions](https://docs.databricks.com/aws/en/large-language-models/ai-functions) in Databricks to deliver batch inference sentiment analysis on your data in only a few lines of code
- Create  a single, simple pipeline to detect sentiment. This pipeline can then be used for managing tables for reporting, ad hoc queries, and/or decision support.
- Create a Genie room so you can explore your sentiment data with natural language interactions 
- Create a dashboard for monitoring sentiment back to the business and drive insights and action


# Introduction

In the previous notebook we used the AI_QUERY function with a custom prompt to analyse our data. This output was stored in the amazon_reviews_sentiment table in such a way that's readily accessible and understood by a human operator.

In this notebook, we'll join this data with item metadata so we can look at key feedback by product, identify those with the worst feedback and analyse key themes and suggestions for improvement.

![](images/Serve.png)

In [0]:
catalog = dbutils.widgets.text("catalog",'mido_edw_dev')
schema = dbutils.widgets.text("schema",'sentiment_analysis')
volume = dbutils.widgets.text("volume",'reviews')

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
volume = dbutils.widgets.get("volume")
transpiledtable = 'amazon_reviews_sentiment'
transpiledtable = f'{catalog}.{schema}.{transpiledtable}'
analysistable = 'Reviews_Analysis'
themestable = 'Themes_Analysis'


#Step 1 - Summarise

In our first step, we'll join our item data with the sentiment data into a single dataset.

Run the cell below to create the view vAllReviewData

In [0]:

spark.sql(f"""CREATE OR REPLACE VIEW {catalog}.{schema}.vAllReviewData
          as    
          select 
          i.title item_name,
          i.parent_asin item_id,
          i.average_rating,
          r.output
          from {catalog}.{schema}.amazon_items i
          inner join {catalog}.{schema}.amazon_reviews_sentiment r 
          on i.parent_asin = r.parent_asin
         """).display()



Run the following cell to persist the review sentiment data in a human readable format. Note the use of get_json_object to read the specific json field provided by the LLM

In [0]:
query = f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{analysistable}
as
SELECT
item_id,
item_name,
average_rating,
get_json_object(output, '$.sentiment') AS sentiment,
get_json_object(output, '$.sentiment_score') AS sentiment_score,
get_json_object(output, '$.key_themes') AS key_themes,
get_json_object(output, '$.suggestions') As Suggestions,
1 as Review_Count
FROM {catalog}.{schema}.vAllReviewData
"""

spark.sql(query)


We can now view the data and check all is as expected. As we can see, the sentiment data has been broken out into specific fields for consumption

In [0]:
spark.sql(f"select * from {catalog}.{schema}.{analysistable}").display()


##Transform
We will also transform our nested themes data to allow us to analyse product by theme also. See below for an example transformation:

In [0]:
query = f"""
          CREATE OR REPLACE TABLE {catalog}.{schema}.{themestable}
          SELECT
            item_id,
            item_name,
            explode(from_json(key_themes, 'array<string>')) AS Key_theme
          FROM {catalog}.{schema}.{analysistable};
          """

spark.sql(query)
spark.sql(f"select * from {catalog}.{schema}.{themestable}").display()


#AI/BI

Along with Genie, Databricks also provides powerful Dashboarding functionality with AI/BI. Please click here to see an example Dashboard that utilises our Sentiment Analysis data.

You can access your example dashboard from the navigation tree on the left, titled "Product Feedback Analysis Dashboard"

![](images/Example Dash.png)